In [5]:
#!pip install PyMuPDF
#!pip install fitz
#!pip install ollama
#!pip install pandas
#!pip install tqdm

In [11]:
import fitz
import ollama
import json
import os
import pandas as pd
from tqdm import tqdm
from IPython.display import display, HTML, clear_output

In [1]:
def pdf_to_str(pdf_filename):
    with fitz.open(pdf_filename) as doc:
        text = ""
        for page in doc:
            text += page.get_text()
    return text    

def get_document_fact(text, query):
    if text is None or text == '':
        return ''
    lst_messages = []

    lst_messages.append({'role': 'user', 'content': 'Next message will contain a CV of a person. \
    You will need to answer a question about this CV that will follow.'})
    lst_messages.append({'role': 'user', 'content': text})


    def add_question(str_query):
        lst_messages.append({'role': 'user', 'content': f"""Given the CV provided above, answer the following question: {str_query} \
        Respond with the complete but shortest possible and direct answer as if you were API. Remove meta words from the answer. \
        Don't say things like "Person phone number is ..." or "Person name is ...". or "The name provided in the CV ..."\
        Just provide the value itself."""})

    add_question(query)

    lst_messages.append(ollama.chat(
        model='mistral',
        messages=lst_messages,
        stream=False,
    )['message'])

    return lst_messages[-1]['content'].strip()

lst_key_query = [
    'What is the person name?',
    'What is the person email?',
    'What is the person title?',
]

lst_fact_query = [
    'What is the person phone?',
    'What languages does person speak?',
    
    'How many years of experience person has?',
    'List the technologies that person has experience with.',
    'List the postitons person worked on in the last 5 years.',
    'Did person work in B level management roles?',
    'Did person work in executive C-level leadership roles?',
    'What education and certifications person has?',
]

def get_document_facts(text, lst_key_query, lst_fact_query):
    dict_doc_key = {
        k: get_document_fact(text, k) for k in lst_key_query
    }
    
    dict_doc_facts = {
        k: get_document_fact(text, k) for k in lst_fact_query
    }
    
    dict_doc_facts['id'] = ', '.join(dict_doc_key.values())
    
    return dict_doc_facts
    
def get_pdf_facts(pdf_filename, lst_key_query, lst_fact_query):
    text = pdf_to_str(pdf_filename)
    dict_facts = get_document_facts(text[:6000], lst_key_query, lst_fact_query)
    return dict_facts
 
lst_doc_filename = os.listdir('input')

lst_dict_res = []
for filename in tqdm(lst_doc_filename):
    dict_res = get_pdf_facts(f"input/{filename}", lst_key_query, lst_fact_query)
    dict_res['filename'] = filename
    lst_dict_res.append(dict_res)
    df_res = pd.DataFrame(lst_dict_res)
    
    clear_output(wait=True)
    display(df_res)

In [2]:
!pip3 freeze > requirements.txt